In [8]:
# from drdid.reg_did import reg_panel_att_if
import pandas as pd, numpy as np

data = pd.read_csv("../csdid_comparing/data/R_panel.csv")
data.head(2)

,treated,age,educ,black,married,nodegree,dwincl,re74,re75,re78,hisp,early_ra,sample,experimental,i_w
0,NaN,46,14,0,1,0,NaN,391.853363,0.000000,0.000000,0,NaN,2,0,0.155489
1,NaN,35,14,0,1,0,NaN,25862.322266,16823.662109,12059.726562,0,NaN,2,0,0.754220


In [9]:
y1 = data['re78'].to_numpy()
y0 = data['re75'].to_numpy()
d = data['experimental'].to_numpy()
w = np.array(data['i_w'])
x = data[['age']].to_numpy()

In [10]:
import numpy as np
from scipy import stats
import numpy as np
import statsmodels.api as sm

lm = sm.WLS
glm = sm.GLM
n_x = np.newaxis
qr_solver = np.linalg.pinv
binomial = sm.families.Binomial()
mean = np.mean

def reg_did_panel(y1, y0, D, covariates, i_weights=None):
    D = np.asarray(D).flatten()
    n = len(D)
    deltaY = np.asarray(y1 - y0).flatten()
    int_cov = np.ones((n, 1))
    
    if covariates is not None:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))
    
    if i_weights is None:
        i_weights = np.ones(n)
    elif np.min(i_weights) < 0:
        raise ValueError("i_weights must be non-negative")
    
    i_weights = i_weights / np.mean(i_weights)
    
    mask = D == 0
    X = int_cov[mask]
    y = deltaY[mask]
    w = i_weights[mask]
    
    # reg_coeff = np.linalg.lstsq(X * w[:, np.newaxis], y * w, rcond=None)[0]
    reg_coeff = lm(y, X, weights=w).fit().params
    print(reg_coeff)
    
    if np.any(np.isnan(reg_coeff)):
        raise ValueError("Outcome regression model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is probably the reason for it.")
    
    out_delta = np.dot(int_cov, reg_coeff)
    w_treat = i_weights * D
    w_cont = i_weights * (1 - D)
    reg_att_treat = w_treat * deltaY
    reg_att_cont = w_cont * out_delta
    eta_treat = np.mean(reg_att_treat) / np.mean(w_treat)
    eta_cont = np.mean(reg_att_cont) / np.mean(w_cont)
    reg_att = eta_treat - eta_cont
    
    weights_ols = i_weights * (1 - D)
    wols_x = weights_ols[:, np.newaxis] * int_cov
    wols_eX = weights_ols[:, np.newaxis] * (deltaY - out_delta)[:, np.newaxis] * int_cov
    XpX_inv = np.linalg.inv(np.dot(wols_x.T, int_cov) / n)
    asy_lin_rep_ols = np.dot(wols_eX, XpX_inv)
    
    inf_treat = (reg_att_treat - w_treat * eta_treat) / np.mean(w_treat)
    print(np.sum(w_treat * eta_treat))
    
    inf_cont_1 = (reg_att_cont - w_cont * eta_cont)
    M1 = np.mean(w_cont[:, np.newaxis] * int_cov, axis=0)
    inf_cont_2 = np.dot(asy_lin_rep_ols, M1)
    inf_control = (inf_cont_1 + inf_cont_2) / np.mean(w_cont)
    
    reg_att_inf_func = (inf_treat - inf_control)
    se_reg_att = np.std(reg_att_inf_func) / np.sqrt(n)
    
    return reg_att, se_reg_att
reg_did_panel(y1, y0, d, x, w)

[6931.2961773  -168.67231538]
252651.52672451953


(475.214004116089, 594.302830865747)

In [11]:
import numpy as np
import statsmodels.api as sm

def drdid_panel(y1, y0, D, covariates, i_weights=None, boot=False, boot_type="weighted", nboot=None, inffunc=False):
    # Convert inputs to numpy arrays
    D = np.asarray(D).flatten()
    n = len(D)
    deltaY = np.asarray(y1 - y0).flatten()
    
    # Add constant to covariate matrix
    if covariates is None:
        int_cov = np.ones((n, 1))
    else:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))
    
    # Weights
    if i_weights is None:
        i_weights = np.ones(n)
    elif np.min(i_weights) < 0:
        raise ValueError("i_weights must be non-negative")
    
    # Normalize weights
    i_weights = i_weights / np.mean(i_weights)
    # print(D.mean())
    
    # Compute the Pscore by MLE
    pscore_model = sm.GLM(D, int_cov, family=sm.families.Binomial(), freq_weights=i_weights)
    pscore_results = pscore_model.fit()
    # print(D.mean())
    # print(pscore_results.summary2())
    if not pscore_results.converged:
        print("Warning: glm algorithm did not converge")
    if np.any(np.isnan(pscore_results.params)):
        raise ValueError("Propensity score model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is a likely reason.")
    ps_fit = pscore_results.predict()
    ps_fit = np.minimum(ps_fit, 1 - 1e-16)
    # print(ps_fit)
    
    # Compute the Outcome regression for the control group using wols
    mask = D == 0
    reg_model = sm.WLS(deltaY[mask], int_cov[mask], weights=i_weights[mask])
    reg_results = reg_model.fit()
    if np.any(np.isnan(reg_results.params)):
        raise ValueError("Outcome regression model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is a likely reason.")
    out_delta = np.dot(int_cov, reg_results.params)
    
    # Compute Traditional Doubly Robust DiD estimators
    w_treat = i_weights * D
    w_cont = i_weights * ps_fit * (1 - D) / (1 - ps_fit)
    dr_att_treat = w_treat * (deltaY - out_delta)
    dr_att_cont = w_cont * (deltaY - out_delta)
    
    eta_treat = np.mean(dr_att_treat) / np.mean(w_treat)
    eta_cont = np.mean(dr_att_cont) / np.mean(w_cont)
    
    dr_att = eta_treat - eta_cont
    
    # Compute influence function
    weights_ols = i_weights * (1 - D)
    wols_x = weights_ols[:, np.newaxis] * int_cov
    wols_eX = weights_ols[:, np.newaxis] * (deltaY - out_delta)[:, np.newaxis] * int_cov
    XpX_inv = np.linalg.inv(np.dot(wols_x.T, int_cov) / n)
    asy_lin_rep_wols = np.dot(wols_eX, XpX_inv)
    
    score_ps = i_weights[:, np.newaxis] * (D - ps_fit)[:, np.newaxis] * int_cov
    Hessian_ps = pscore_results.cov_params() * n
    asy_lin_rep_ps = np.dot(score_ps, Hessian_ps)
    
    inf_treat_1 = dr_att_treat - w_treat * eta_treat
    M1 = np.mean(w_treat[:, np.newaxis] * int_cov, axis=0)
    inf_treat_2 = np.dot(asy_lin_rep_wols, M1)
    inf_treat = (inf_treat_1 - inf_treat_2) / np.mean(w_treat)
    
    inf_cont_1 = dr_att_cont - w_cont * eta_cont
    M2 = np.mean(w_cont[:, np.newaxis] * (deltaY - out_delta - eta_cont)[:, np.newaxis] * int_cov, axis=0)
    inf_cont_2 = np.dot(asy_lin_rep_ps, M2)
    M3 = np.mean(w_cont[:, np.newaxis] * int_cov, axis=0)
    inf_cont_3 = np.dot(asy_lin_rep_wols, M3)
    inf_control = (inf_cont_1 + inf_cont_2 - inf_cont_3) / np.mean(w_cont)
    
    dr_att_inf_func = inf_treat - inf_control
    
    return dr_att, dr_att_inf_func

drdid_panel(y1, y0, d, None, w)

(475.21400411608613,
 array([  423.37312598,  9421.37209022, 14394.03516237, ...,
         2231.7013017 , 25987.48210317,  1468.3600495 ]))

In [12]:
import numpy as np
import statsmodels.api as sm

nopanel = pd.read_csv("../csdid_comparing/data/R_nopanel_reg.csv")
# y = nopanel['y']
# post = nopanel['post']
# D = nopanel['d']
# w = nopanel['i_w']

def reg_did_rc(y, post, D, covariates, i_weights=None):
    D = np.asarray(D).flatten()
    post = np.asarray(post).flatten()
    n = len(D)
    y = np.asarray(y).flatten()
    i_weights = np.asarray(i_weights).flatten()
    int_cov = np.ones((n, 1))
    
    if covariates is not None:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))
    
    if i_weights is None:
        i_weights = np.ones(n)
    elif np.min(i_weights) < 0:
        raise ValueError("i_weights must be non-negative")
    
    i_weights = i_weights / np.mean(i_weights)
    
    # Pre-treatment regression
    mask_pre = (D == 0) & (post == 0)
    X_pre = int_cov[mask_pre]
    y_pre = y[mask_pre]
    w_pre = i_weights[mask_pre]
    model_pre = sm.WLS(y_pre, X_pre, weights=w_pre)
    results_pre = model_pre.fit()
    reg_coeff_pre = results_pre.params
    
    if np.any(np.isnan(reg_coeff_pre)):
        raise ValueError("Outcome regression model coefficients have NA components. \n Multicollinearity of covariates is probably the reason for it.")
    
    out_y_pre = np.dot(int_cov, reg_coeff_pre)
    
    # Post-treatment regression
    mask_post = (D == 0) & (post == 1)
    X_post = int_cov[mask_post]
    y_post = y[mask_post]
    w_post = i_weights[mask_post]
    model_post = sm.WLS(y_post, X_post, weights=w_post)
    results_post = model_post.fit()
    reg_coeff_post = results_post.params
    
    if np.any(np.isnan(reg_coeff_post)):
        raise ValueError("Outcome regression model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is probably the reason for it.")
    
    out_y_post = np.dot(int_cov, reg_coeff_post)
    
    w_treat_pre = i_weights * D * (1 - post)
    w_treat_post = i_weights * D * post
    w_cont = i_weights * D
    reg_att_treat_pre = w_treat_pre * y
    reg_att_treat_post = w_treat_post * y
    reg_att_cont = w_cont * (out_y_post - out_y_pre)
    eta_treat_pre = np.mean(reg_att_treat_pre) / np.mean(w_treat_pre)
    eta_treat_post = np.mean(reg_att_treat_post) / np.mean(w_treat_post)
    eta_cont = np.mean(reg_att_cont) / np.mean(w_cont)
    reg_att = (eta_treat_post - eta_treat_pre) - eta_cont
    
    weights_ols_pre = i_weights * (1 - D) * (1 - post)
    # print(weights_ols_pre.reshape((n, 1)))
    wols_x_pre = weights_ols_pre[:, np.newaxis] * int_cov
    wols_eX_pre = weights_ols_pre[:, np.newaxis] * (y - out_y_pre)[:, np.newaxis] * int_cov
    XpX_inv_pre = np.linalg.inv(np.dot(wols_x_pre.T, int_cov) / n)
    asy_lin_rep_ols_pre = np.dot(wols_eX_pre, XpX_inv_pre)
    
    weights_ols_post = i_weights * (1 - D) * post
    wols_x_post = weights_ols_post[:, np.newaxis] * int_cov
    wols_eX_post = weights_ols_post[:, np.newaxis] * (y - out_y_post)[:, np.newaxis] * int_cov
    XpX_inv_post = np.linalg.inv(np.dot(wols_x_post.T, int_cov) / n)
    asy_lin_rep_ols_post = np.dot(wols_eX_post, XpX_inv_post)
    
    inf_treat_pre = (reg_att_treat_pre - w_treat_pre * eta_treat_pre) / np.mean(w_treat_pre)
    inf_treat_post = (reg_att_treat_post - w_treat_post * eta_treat_post) / np.mean(w_treat_post)
    inf_treat = inf_treat_post - inf_treat_pre
    
    inf_cont_1 = (reg_att_cont - w_cont * eta_cont)
    M1 = np.mean(w_cont[:, np.newaxis] * int_cov, axis=0)
    inf_cont_2_post = np.dot(asy_lin_rep_ols_post, M1)
    inf_cont_2_pre = np.dot(asy_lin_rep_ols_pre, M1)
    inf_control = (inf_cont_1 + inf_cont_2_post - inf_cont_2_pre) / np.mean(w_cont)
    
    reg_att_inf_func = (inf_treat - inf_control)
    se_reg_att = np.std(reg_att_inf_func) / np.sqrt(n)
    
    return reg_att, reg_att_inf_func, se_reg_att
# reg_did_rc(y, post, D, None, w)

In [13]:
import numpy as np
from scipy import stats
import statsmodels.api as sm

def ipw_did_panel(y1, y0, D, covariates, i_weights=None):
    # Convert inputs to numpy arrays
    y1 = np.asarray(y1)
    y0 = np.asarray(y0)
    D = np.asarray(D)
    
    # Sample size
    n = len(D)
    
    # Generate deltaY
    deltaY = y1 - y0
    
    # Add constant to covariate vector
    if covariates is None:
        int_cov = np.ones((n, 1))
    else:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))
    
    # Weights
    if i_weights is None:
        i_weights = np.ones(n)
    elif np.min(i_weights) < 0:
        raise ValueError("i_weights must be non-negative")
    
    # Normalize weights
    i_weights = i_weights / np.mean(i_weights)
    
    # Pscore estimation (logit) and its fitted values
    PS = sm.GLM(D, int_cov, family=sm.families.Binomial(), freq_weights=i_weights).fit()
    if not PS.converged:
        print("Warning: glm algorithm did not converge")
    if np.any(np.isnan(PS.params)):
        raise ValueError("Propensity score model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is a likely reason.")
    ps_fit = PS.predict()
    ps_fit = np.minimum(ps_fit, 1 - 1e-16)
    
    # Compute IPW estimator
    w_treat = i_weights * D
    w_cont = i_weights * ps_fit * (1 - D) / (1 - ps_fit)
    att_treat = w_treat * deltaY
    att_cont = w_cont * deltaY
    eta_treat = np.mean(att_treat) / np.mean(i_weights * D)
    eta_cont = np.mean(att_cont) / np.mean(i_weights * D)
    ipw_att = eta_treat - eta_cont
    
    # Get the influence function to compute standard error
    score_ps = i_weights[:, np.newaxis] * (D - ps_fit)[:, np.newaxis] * int_cov
    Hessian_ps = PS.cov_params() * n
    asy_lin_rep_ps = np.dot(score_ps, Hessian_ps)
    
    # Get the influence function of control component
    att_lin1 = att_treat - att_cont
    mom_logit = np.mean(att_cont[:, np.newaxis] * int_cov, axis=0)
    att_lin2 = np.dot(asy_lin_rep_ps, mom_logit)
    
    # Get the influence function of the IPW estimator
    att_inf_func = (att_lin1 - att_lin2 - i_weights * D * ipw_att) / np.mean(i_weights * D)
    print(np.std(att_inf_func) / np.sqrt(n))
    return ipw_att, att_inf_func
ipw_did_panel(y1, y0, d, None, w)

594.302814443883


(475.21400407689976,
 array([  423.36996588,  9421.35676191, 14394.01780432, ...,
         2231.68464404, 25987.47241037,  1468.3490895 ]))

In [19]:
def std_ipw_did_panel(y1, y0, D, covariates, i_weights = None):
    D = np.asarray(D).flatten()
    n = len(D)
    delta_y = np.asarray(y1 - y0).flatten()
    int_cov = np.ones((n, 1))
    
    if covariates is not None:
        covariates = np.asarray(covariates)
        if np.all(covariates[:, 0] == 1):
            int_cov = covariates
        else:
            int_cov = np.column_stack((np.ones(n), covariates))
    
    if i_weights is None:
        i_weights = np.ones(n)
    elif np.min(i_weights) < 0:
        raise ValueError("i_weights must be non-negative")
    
    i_weights = i_weights / np.mean(i_weights)
    pscore_model = sm.GLM(D, int_cov, family=sm.families.Binomial(), freq_weights=i_weights)
    pscore_results = pscore_model.fit()
    # print(D.mean())
    # print(pscore_results.summary2())
    if not pscore_results.converged:
        print("Warning: glm algorithm did not converge")
    if np.any(np.isnan(pscore_results.params)):
        raise ValueError("Propensity score model coefficients have NA components. \n Multicollinearity (or lack of variation) of covariates is a likely reason.")
    ps_fit = pscore_results.predict()
    ps_fit = np.minimum(ps_fit, 1 - 1e-16)

    w_treat = i_weights * D
    w_cont = i_weights * ps_fit * (1 - D) / (1 - ps_fit)
    
    att_treat = w_treat * delta_y
    att_cont = w_cont * delta_y

    eta_treat = mean(att_treat) / mean(w_treat)
    eta_cont = mean(att_cont) / mean(w_cont)

    ipw_att = eta_treat - eta_cont

    score_ps = i_weights[:, np.newaxis] * (D - ps_fit)[:, np.newaxis] * int_cov
    Hessian_ps = pscore_results.cov_params() * n
    asy_lin_rep_ps = np.dot(score_ps, Hessian_ps)

    inf_treat = (att_treat - w_treat * eta_treat) / mean(w_treat)
    inf_cont_1 = att_cont - w_cont * eta_cont
    pre_m2 = w_cont * (delta_y - eta_cont)
    M2 = np.mean(pre_m2[:, np.newaxis] * int_cov, axis = 0)
    print(M2)
    inf_cont_2 = np.dot(asy_lin_rep_ps, M2)

    inf_control = (inf_cont_1 + inf_cont_2) / np.mean(w_cont)
    att_inf_func = inf_treat - inf_control
    print(np.std(att_inf_func) / np.sqrt(n))
    return ipw_att, att_inf_func

std_ipw_did_panel(y1, y0, d, None, w)
    

[1.34605216e-14]
594.3028308657471


(475.2140041160869,
 array([  423.37312598,  9421.37209022, 14394.03516237, ...,
         2231.7013017 , 25987.48210317,  1468.3600495 ]))